![Local image](./images/silver/06_FA-01.png)

지금까지 다뤘던 타뷸러 형태의 문제는 상태와 액션이 몇가지로 제한되어 있다. 하지만 실제 필드에서 다뤄야 하는 문제는 그렇게 단순하지 않다. 실제 문제에 강화학습을 적용하기 위해서는 지금까지 다뤘던 방법들을 scale-up할 수 있는 접근이 필요하다.

이 장에서는 가치함수(value function)를 함수 근사(function approximation)로 표현하여 정책 평가, 컨트롤 등을 하는 방법에 대해 다룬다.


![Local image](./images/silver/06_FA-02.png)

'Incremental Methods'에서는 새로운 데이터(에피소드)를 관찰할 때마다 모델 파라미터를 업데이트 하는 방식(on-line learning)에 대해 알아볼 것이고, 'Batch Methods'에서는 특정 시점까지 관찰한 모든 데이터를 한꺼번에 이용해 가장 실제가 가까운 가치함수를 찾는 방법에 대해 알아볼 것이다.

![Local image](./images/silver/06_FA-03.png)

![Local image](./images/silver/06_FA-05.png)

###### 현실에서 다뤄야 하는 문제는 타뷸러 형태로 다루기 어렵다. 

백가몬 게임에서는 $10^{20}$개, 바둑에서는 $10^{170}$개 정도로 많은 상태를 다뤄야 한다. 또한 헬리콘터 컨트롤 문제에서는 심지어 연속적인 상태 공간을 다뤄야 한다. 앞서 다뤘던 방법에서는 각 상태를 테이블 형태로 나타내고 관측한 에피소드에 따라 각 에피소드의 가치 값을 바꿨었는데, 상태가 셀수 없이 많은 문제는 더 이상 이런 접근으로 다룰 수 없다.

다수의 상태를 쉽게 다루기 위해 각 상태(+행동)에 대응하는 가치 값을 출력하는 함수가 있다고 가정하고, 에피소드 정보를 이용해 이 함수를 근사하는 방법을 사용한다. 

![Local image](./images/silver/06_FA-06.png)

지금까지의 문제는 각 상태에 대한 가치($v(s)$)나 상태-행동에 대한 가치 $Q(s, a)$를 테이블 형태로 표현했었다. 

하지만 상태가 매우 많은 문제를 테이블 형태로 표현하려면 필요로 하는 메모리 양이 너무 많아지고, 각 상태들을 모두 학습시켜야 하므로 그 시간 또한 오래 걸리는 문제점이 있다.

결국 알고 싶은 것은 실제 가치에 대응하는 가치값을 표현하는 어떤 함수 $v_{\pi}(s)$인데, 이를 어떤 가중치 벡터 $\mathbf{w}$를 모수로 하는 모수 함수(parametric function) $\hat{v}(s, \mathbf{w})$로 근사하는 것이다. 이 함수는 회귀 모형이나 신경망 혹은 의사결정 나무 등 다양한 지도학습 모형이 될 수 있다. 예를들어 이 함수가 신경망이라면 $\mathbf{w}$는 그 신경망으로 학습된 가중치 벡터가 될 것이다. 이 가중치의 수는 보통 가능한 상태의 수 보다 훨씬 작은 수준이 될 것이다.

이런 함수 근사를 상태-행동에 대응하는 방식으로 설계할수도 있다. 즉 $q_{\pi}(s, a)$를 근사하는 함수 $\hat{q}_{\pi}(s, a, \mathbf{w})$를 학습시키는 것이다.

이렇게 함수 근사로 가치함수를 표현하게 되면 관측하지 않았던 상태(혹은 상태-행동)에 대응하는 가치 또한 근사적으로 알 수 있게 된다.

$\hat{v}(s, \mathbf{w})$가 $v_{\pi}(s)$에 가까워지도록 $\mathbf{w}$를 업데이트(학습)해야하는데, 지금까지 배웠던 MC나 TD와 같은 방법을 여기에 사용하게 된다.

![Local image](./images/silver/06_FA-07.png)

가치 함수를 근사하는 방식(구조)을 3가지로 나눠볼 수 있다.      
첫번째 방식은 상태 $s$를 입력하면 그 상태에 해당하는 가치를 반환하는 함수 $\hat{v}(s, \mathbf{w})$로 근사하는 것이고(TRPO, PPO, CPO),       
두번째 방식은 상태-행동 쌍 $(s, a)$에 대한 가치를 반환하는 함수 $\hat{q}(s, \mathbf{w})$로 근사하는 것이고(DDPG),     
세번째 방식은 상태 $s$를 입력하면 그 상태에서 가능한 모든 행동에 해당하는 가치를 반환하는 함수 $\hat{q}(s, a_1, \mathbf{w}), \cdots, \hat{q}(s, a_m, \mathbf{w})$로 근사하는 것이다.(DQN)

![Local image](./images/silver/06_FA-08.png)

함수 근사를 위한 다양한 방법들이 있다. 

![Local image](./images/silver/06_FA-09.png)

다양한 함수 근사 방법들중 이 강의에서는 선형 가중합 형태의 모형(회귀 모형 형태)이나 신경망과 같이 미분 가능한 함수 근사 방법들을 사용할 것이다.(이런 방법들은 gradient를 이용해 바로 모수를 업데이트 할 수 있는 장점이 있다.) 

사실 사용할 수 있는 지도학습 방법에는 몇가지 제약이 더 있다. 비정상(non-stationary)이고 각 샘플간에 iid 가정이 성립하지 않는(non-iid) 데이터를 이용해 학습이 가능한 방법들이어야 한다. 정상성을 이란 시간에 따라 확률분포가 변하지 않는 특성을 의미하는데, 강화학습에서는 일정한 time-step마다 정책을 업데이트 하고 그에 따라 다음번에 추출할 상태-행동의 분포가 달라지기 때문에 정상성을 갖지 않는다. 또한 지금의 상태-행동이 다음 시점의 상태-행동에 영향을 주기 때문에 iid(independent identically distribution)이지도 않다.

![Local image](./images/silver/06_FA-10.png)

![Local image](./images/silver/06_FA-11.png)

우선 Gradient Descent는 $\mathbf{w}$를 모수로 하는 미분 가능한 함수 $J(\mathbf{w})$를 최소로 하는 $\mathbf{w}$를 찾기 위한 방법으로서, gradient $\bigtriangledown_{\mathbf{w}}J(\mathbf{w})$의 반대방향으로 $-1/2 \alpha \bigtriangledown_{\mathbf{w}}J(\mathbf{w})$만큼 $\mathbf{w}$를 반복적으로 수정한다.

![Local image](./images/silver/06_FA-12.png)

어떤 목적 함수 $J$를 최소하는 방식으로 가치함수를 근사하기 위해 아래와 같이 진짜 가치함수 $v_{\pi}(S)$와 근사 가치함수 $\hat{v}(S, \mathbf{w})$와의 차이 제곱의 기댓값 형태의 목적 함수 $J(\mathbf{w})$를 생각해볼 수 있다.
$$J(\mathbf{w}) = \mathbb{E}_{\pi}\Big[ \big( v_{\pi}(S) - \hat{v}(S, \mathbf{w})  \big)^2 \Big]$$

Gradient Descent는 전체 데이터 샘플에 대한 오차 제곱 $\big( v_{\pi}(S) - \hat{v}(S, \mathbf{w})  \big)^2$들을 구하고 이들의 기댓값을 구하고 이를 $\mathbf{w}$에 사용한다.
반면 **Stochastic** gradient descent는 각 샘플에 대한 오차 제곱을 이용해 바로 $\mathbf{w}$를 업데이트 한다.

(위 설정에서는) 근사의 타겟이 되는 $v_{\pi}(S)$는 $\mathbf{w}$에 와 무관하므로 gradient $J(\mathbf{w})$는 단순히 $\bigtriangledown_{\mathbf{w}}\hat{w}(S, \mathbf{w})$이다.
만약 근사의 타겟이 $\mathbf{w}$에 대한 함수라면 gradient가 달라져야하는데, 이때에도 gradient로 $\bigtriangledown_{\mathbf{w}}\hat{w}(S, \mathbf{w})$를 사용하는 방법이 semi stochastic gradient descent이다.

![Local image](./images/silver/06_FA-13.png)

가치함수를 $\hat{v}(S, \mathbf{w})$로 근사한다면, $S$자리에 들어갈 정보가 필요하다. 이 정보는 각 상태의 특성을 표현하는 값들이며 피쳐 벡터이다. 예를들어 로봇의 움직임을 컨트롤 한다면 로봇과 장애물과의 상대 위치를 표현하는 각도와 거리가 각각 $x_1(S)$과 $x_2(S)$라 할 수 있다.

![Local image](./images/silver/06_FA-14.png)

가치함수를 근사하는 가장 단순한 방법은 가치함수를 $\mathbf{x}(S)^T \mathbf{w}$와 같이 피쳐들의 가중합으로 표현하는 것이다. 이 경우 (MSE를 이용한) 목적 함수는 아래와 같은 형태가 될 것이다.
$$J(\mathbf{w}) = \mathbb{E}_{\pi} \Big[ \big( v_{\pi}(S) - \mathbf{x}(S)^T \mathbf{w} \big)^2 \Big]$$
위 목적함수는 $w$에 대한 2차식으로서 $J(\mathbf{w})$의 극소점이 반드시 하나 존재하기 때문에 최적화가 매우 쉽다.

만약 이를 SGD형태로 구현하는 경우 gradient가 $x(S)$이므로 그 계산 또한 매우 단순하다.

![Local image](./images/silver/06_FA-15.png)

그런데 초반부터 다뤘던 테이블 룩업 방법은 함수 근사의 특별한 경우라 할 수 있다. 예를들어 A, B, C 3개의 상태가 존재하는 환경에서 10개의 에피소드를 관측했을 때까지 각 상태의 가치가 $v_{\pi}(A)=3$, $v_{\pi}(B)=1$, $v_{\pi}(C)=5$라고 해보자. 이런 단순한 테이블 룩업을 선형 함수 근사로 표현할 수 있다.

즉 상태에 대한 피쳐 벡터를 아래와 같이 나타낼 수 있다.
$$\begin{align}
\mathbf{x}(A)=(1, 0, 0)^T \\
\mathbf{x}(B)=(0, 1, 0)^T \\
\mathbf{x}(C)=(0, 0, 1)^T \\
\end{align}$$

또한 이에 대응하는 가중치 벡터 $\mathbf{w}$를 아래와 같이 나타낼 수 있다.
$$\mathbf{w} = (3, 1, 5)$$

이때 각 상태의 가치 값을 아래와 같이 표현할 수 있다.

$$\begin{align}
\hat{v}(A, \mathbf{w}) &= \mathbf{x}(A)^T \mathbf{w} = (1, 0, 0) \cdot (3, 1, 5)^T = 3 \\
\hat{v}(B, \mathbf{w}) &= \mathbf{x}(B)^T \mathbf{w} = (0, 1, 0) \cdot (3, 1, 5)^T = 1 \\
\hat{v}(C, \mathbf{w}) &= \mathbf{x}(C)^T \mathbf{w} = (0, 0, 1) \cdot (3, 1, 5)^T = 5
\end{align}$$

![Local image](./images/silver/06_FA-16.png)

앞서의 논의에서는 실제 가치함수 $v_{\pi}(s)$가 주어지고, 이것을 근사의 타겟으로 하여 근사 함수 $\hat{v}(S, \mathbf{w})$와 이것과의 차이를 목적함수에 이용했었다. 그런데 실전에서는 실제 가치함수 $v_{\pi}(s)$가 주어지지 않는다. 대신 경험(에피소드)을 통해 알게된 조금 더 실제 가치함수에 근접한 어떤 값 혹은 함수를 근사의 타겟으로 이용한다.

예를들어 MC에서는 각 에피소드를 통해 계산된 반환값 $G_t$를 타겟으로 한다. 예를들어 에피소드를 끝까지 관찰한 후 알게되는 반환값 $G_t$과 $t$번째 time-step에서 근사한 가치값 $\hat{v}(S_t, \mathbf{w})$과의 차이($G_t - \hat{v}(S_t, \mathbf{w})$)를 이용한다.       
마찬가지로 TD(0)에서는 1-step TD return ($R_{t+1} + \gamma \hat{v}(S_{t+1}, \mathbf{w})$)을 타겟으로 하며      
TD($\lambda$)에서는 ($\lambda$+1)-step TD return ($G_t^{\lambda}$)을 타겟으로 한다.

![Local image](./images/silver/06_FA-17.png)

MC를 이용한 가치함수 근사에서는 $G_t$를 타겟으로 $\mathbf{w}$를 업데이트 한다.

$G_t$가 $v_{\pi}(S_t)$에 대한 불편(unbiased) 추정량이므로 이런 업데이트는 결국 local optimun으로 수렴한다. (선형 근사를 쓸 경우 global optimum으로 수렴)

![Local image](./images/silver/06_FA-18.png)

TD(0)를 이용할 때는 $R + \gamma \hat{v}(S', \mathbf{w})$를 타겟으로 한다.

TD-target은 $v_{\pi}(S_t)$에 대한 편향(biased)추정량이다. 그럼에도 선형 근사를 하면 global optimun에 가까이 수렴한다.

![Local image](./images/silver/06_FA-20.png)

TD($\lambda$)를 이용할 때는 $G_t^{\lambda}$(n-step TD return)를 타겟으로 한다.

# Control...

![Local image](./images/silver/06_FA-21.png)

![Local image](./images/silver/06_FA-22.png)

![Local image](./images/silver/06_FA-23.png)

$\mathbf{x}(S, A)$는 상태-행동 쌍에 대한 feature vector를 의미

![Local image](./images/silver/06_FA-24.png)

![Local image](./images/silver/06_FA-25.png)

![Local image](./images/silver/06_FA-26.png)

![Local image](./images/silver/06_FA-27.png)

![Local image](./images/silver/06_FA-28.png)

![Local image](./images/silver/06_FA-29.png)

![Local image](./images/silver/06_FA-30.png)

![Local image](./images/silver/06_FA-31.png)

![Local image](./images/silver/06_FA-32.png)

![Local image](./images/silver/06_FA-33.png)

![Local image](./images/silver/06_FA-34.png)

(Stochastic) Gradient Descent는 연산량이 적고 속도가 빠르지만, 한번 업데이트에 사용된 샘플은 다음 업데이트에 고려하지 않기 때문에 데이터를 효과적으로 사용하지 못한다. 반면 배치 방식은 업데이트 시점까지 쌓인 데이터를 모두 이용해 $\mathbf{w}$를 업데이트 하기 때문에 더 많은 연산량이 필요하지만 데이터를 보다 효율적으로 사용하여 솔루션을 찾을 수 있다.

![Local image](./images/silver/06_FA-35.png)

어떤 '모수함수 $\hat{v}(s, \mathbf{w})$'와 '추론의 근거가 되는 데이터 $\mathcal{D}$'가 주어졌을 때, 최소자승법(Least Square)은 오차 제곱의 기댓값을 최소화 하는 파라미터 $\mathbf{w}$를 찾는 방법이다.

![Local image](./images/silver/06_FA-37.png)

그렇다면 최소자승법의 해(solution)을 어떻게 찾을 수 있나?

다양한 방법중 하나가 'SGD with Experience Replay'이다.      
우선 $\langle s_i, v_i^{\pi}\rangle$ 형태의 데이터를 모두 저장한 후,     
여기서 랜덤하게 샘플을 뽑아 SGD 업데이트를 하는 것이다. 

원래 time-step 마다 관찰하는 $\langle s_i, v_i^{\pi}\rangle$는 이전 이후의 데이터가 서로 영향을 주기 때문에 iid가 아니다. 하지만 위와 같이 전체 데이터에서 랜덤하게 데이터를 추출할 경우 샘플간 상관관계가 깨지기 때문에 iid이고, SGD로도 최소자승법의 해를 구할 수 있다.

![Local image](./images/silver/06_FA-38.png)

DQN에 'Experience Replay'를 이용한다면...

현재의 정책 $\pi$에서 $(s_t, a_t, r_{t+1}, s_{t+1})$들을 생성하고 이 데이터들을 $\mathcal{D}$라 하자. 그리고 $\mathcal{D}$에서 $(s_t, a_t, r_{t+1}, s_{t+1})$를 랜덤하게 추출한다. 수 time-step 이전의 가중치 $w_i^-$를 이용해 계산한 Q-value 값을 타겟으로 $w_i$를 업데이트 한다.


![Local image](./images/silver/06_FA-39.png)

위 구조의 DQN을 여러 게임에 적용(단 네트워크 구조와 hyperparameter는 고정)

![Local image](./images/silver/06_FA-40.png)

여러 게임에 적용 했을 때 수직선 좌측은 인간보다 잘하는 경우이고, 우측은 인간보다 못한 케이스

![Local image](./images/silver/06_FA-41.png)

여러 DQN 방법들을 써 본 결과 Replay를 사용한 Fixed Q-learning이 가장 성능 좋았다.

![Local image](./images/silver/06_FA-42.png)

Experience Replay는 Batch + Stochastic한 방법이다. 하지만 이 방식은 $w$ 업데이트를 여러번해야하는 단점이 있다.

Batch 방식으로 $w$에 대한 닫힌해(closed solution)을 바로 구할 수 있는 방법도 있다. 물론 이런 방식이 적용되기 위해서는 많은 제약조건이 필요한데, 여기서는 선형 최소자승법으로 논의를 한정하며, 근사 함수가 선형 가중합 $\hat{v}(s, \mathbf{w}) = \mathbf{x}(s)^T \mathbf{w}$이라면 $\mathbf{w}$에 대한 해석적 해를 바로 구할 수 있다.

![Local image](./images/silver/06_FA-43.png)

이때의 해는 위와 같이 두개의 행렬 곱으로 구해지며, $\mathbf{w}$가 수렴하면 $\Delta\mathbf{w}$가 0이 된다는 것을 이용한다. 이렇게 구해진 해는 역함수가 포함되므로 연산량이 이전에 논했던 방법에 비해 늘어난다. 대략 상태에 대한 feature 벡터의 길이가 $N$이라면 $\mathcal{O}(N^2)$ 정도의 연산량이 필요하다.

한방에 바로! $\mathbf{w}$를 구할 수 있는 장점이 있지만 $N$이 크다면 실용적이지 않고 선형 근사에서만 적용 가능하다.

![Local image](./images/silver/06_FA-44.png)

MC, TD, TD($\lambda$) 각각에 대한 선형 근사의 닫힌 해를 구해볼 수 있다. 각각 **LS**MC, **LS**TD, **LS**TD($\lambda$)라 부르며, 타겟이 상이하다.

![Local image](./images/silver/06_FA-45.png)

![Local image](./images/silver/06_FA-46.png)

여러 알고리즘의 수렴 특성을 살펴보면 **LS**들은 비선형 근사에는 적용할 수 없으며, TD에 비해 LSTD가 더 나은 수렴 특성을 갖는다.

---

# Control...

![Local image](./images/silver/06_FA-47.png)

![Local image](./images/silver/06_FA-48.png)

![Local image](./images/silver/06_FA-49.png)

![Local image](./images/silver/06_FA-50.png)

![Local image](./images/silver/06_FA-51.png)

![Local image](./images/silver/06_FA-52.png)

![Local image](./images/silver/06_FA-53.png)

![Local image](./images/silver/06_FA-54.png)

![Local image](./images/silver/06_FA-55.png)

![Local image](./images/silver/06_FA-56.png)